# ☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda

Este notebook introduce un flujo moderno de datos en AWS con almacenamiento en S3, transformación con Glue (PySpark), consulta con Athena (SQL sobre S3) y orquestación con eventos/Lambda. Incluye ejemplos de código con `boto3` y prácticas recomendadas.

## Requisitos y Notas de Ejecución

- Para ejecutar código real de AWS necesitas credenciales configuradas en tu entorno (`AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION`).
- Nunca subas credenciales al repositorio. Usa variables de entorno o perfiles de AWS CLI.
- Este notebook es auto-contenido con bloques que pueden ejecutarse si tu entorno ya tiene permisos.
- Alternativa local: usar LocalStack para simular servicios AWS en tu máquina.

### ☁️ **AWS Cloud: Ecosistema para Data Engineering**

**Stack Moderno de AWS para Datos:**

1. **S3 (Simple Storage Service)**: Data Lake base
   - Almacenamiento ilimitado con 99.999999999% durabilidad (11 nines)
   - Pricing: Pay-per-use (~$0.023/GB/mes en Standard)
   - Clases de almacenamiento: Standard → Infrequent Access → Glacier (archival)

2. **Glue**: Servicio ETL serverless con PySpark
   - Auto-escalado de workers (DPU - Data Processing Units)
   - Glue Catalog: Metastore centralizado (Hive-compatible)
   - Crawlers: Descubrimiento automático de schemas

3. **Athena**: Motor SQL interactivo sobre S3
   - Basado en Presto, consulta Parquet/ORC/CSV sin infraestructura
   - Pricing: $5 por TB escaneado (optimizar con particiones)

4. **Redshift**: Data Warehouse MPP (Massively Parallel Processing)
   - Columnar storage con compresión
   - COPY desde S3, UNLOAD hacia S3

5. **EMR (Elastic MapReduce)**: Clusters Spark/Hadoop administrados
   - Spot instances para reducir costos 70-90%
   - Notebooks Jupyter integrados

**Arquitectura Lambda (Batch + Streaming):**
```
Fuentes → [Kinesis/Kafka] → S3 Raw → [Glue/EMR] → S3 Curated → [Athena/Redshift] → BI
                ↓
           Lambda (Near Real-Time)
```

**IAM Best Practices:**
- Roles > Users (principio de menor privilegio)
- Políticas: `s3:GetObject`, `s3:PutObject` (granular por bucket/prefix)
- No hardcodear credenciales: usar IAM Roles, AWS Secrets Manager

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 1. S3: Data Lake Básico

### 🗄️ **S3: Fundamentos del Data Lake**

**Conceptos Core:**

- **Bucket**: Contenedor global único (namespace global AWS)
  - Nombre: `mi-data-lake-123456` (minúsculas, números, guiones)
  - Región: `us-east-1`, `eu-west-1`, etc.

- **Prefix (Key)**: Jerarquía lógica simulada (no son carpetas reales)
  ```
  s3://bucket/raw/ventas/2025/10/ventas_2025_10_30.csv
           │     └─────┬─────┘ └──┬──┘ └────┬────┘
         Bucket      Prefix    Partición   Archivo
  ```

- **Versionamiento**: Protección contra eliminación accidental
  - Enabled: Cada PUT crea nueva versión con ID único
  - Lifecycle policies: Migrar versiones antiguas a Glacier

**Patrones de Organización:**

1. **Medallion Architecture (Databricks):**
   ```
   /bronze/    ← Datos crudos (raw, sin transformar)
   /silver/    ← Cleaned & validated
   /gold/      ← Business-level aggregates
   ```

2. **Por Dominio:**
   ```
   /sales/raw/, /sales/curated/
   /customers/raw/, /customers/curated/
   ```

3. **Particionamiento Hive-style:**
   ```
   /year=2025/month=10/day=30/data.parquet
   ```
   - Athena/Glue entienden automáticamente las particiones
   - Reduce escaneo en queries: `WHERE year=2025 AND month=10`

**Operaciones con boto3:**
- `put_object()`: Upload inline (para archivos pequeños <5GB)
- `upload_file()`: Upload desde disco local (multipart automático)
- `list_objects_v2()`: Listar con paginación (max 1000 por request)

**Costos:**
- PUT/COPY/POST/LIST: $0.005 per 1,000 requests
- GET/SELECT: $0.0004 per 1,000 requests
- Data transfer OUT: $0.09/GB (dentro de AWS: gratis)

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
import os, json, io
import pandas as pd
# import boto3  # Descomenta si tienes credenciales configuradas

BUCKET = 'mi-data-lake-demo-123456'  # Cambia por un nombre único global
PREFIX_RAW = 'raw/ventas/'
PREFIX_CURATED = 'curated/ventas/'

print('👆 Define el bucket y prefijos antes de ejecutar contra AWS')

### 1.1 Crear bucket (opcional)

In [ ]:
# s3 = boto3.client('s3')
# try:
#     s3.create_bucket(Bucket=BUCKET, CreateBucketConfiguration={'LocationConstraint': 'us-east-1'})
#     print('✅ Bucket creado')
# except s3.exceptions.BucketAlreadyOwnedByYou:
#     print('ℹ️ Bucket ya existe')
# except Exception as e:
#     print('❌ Error creando bucket:', e)

### 1.2 Subir dataset de ejemplo a S3

In [ ]:
# df = pd.read_csv('../../datasets/raw/ventas.csv')
# csv_bytes = df.to_csv(index=False).encode('utf-8')
# s3.put_object(Bucket=BUCKET, Key=PREFIX_RAW + 'ventas_2025_10.csv', Body=csv_bytes)
# print('📤 Archivo subido a S3')

## 2. Glue: Transformaciones con PySpark (Job Script)

### 🔧 **AWS Glue: ETL Serverless con PySpark**

**Componentes de Glue:**

1. **Glue Data Catalog:**
   - Metastore centralizado (bases de datos + tablas)
   - Compatible con Hive Metastore (usado por Athena, EMR, Redshift Spectrum)
   - Schema discovery con Crawlers

2. **Glue Crawlers:**
   - Escanean S3 y detectan formato/schema automáticamente
   - Actualizan el Catalog con nuevas particiones
   - Schedule: Cron expressions (`cron(0 0 * * ? *)` = diario a medianoche)

3. **Glue Jobs (ETL Scripts):**
   - Python Shell (para scripts ligeros)
   - PySpark (para transformaciones distribuidas)
   - DPU (Data Processing Unit): 1 DPU = 4 vCPU + 16GB RAM
   - Pricing: $0.44 por DPU-hora

**GlueContext vs SparkContext:**
```python
# SparkContext: Estándar PySpark
sc = SparkContext()

# GlueContext: Extensión AWS con métodos adicionales
glueContext = GlueContext(sc)
glueContext.create_dynamic_frame_from_catalog()  # Lee desde Catalog
glueContext.write_dynamic_frame.from_options()    # Escribe con conversiones
```

**DynamicFrame vs DataFrame:**
- **DataFrame (Spark)**: Tipado estricto, requiere schema consistente
- **DynamicFrame (Glue)**: Schema flexible, maneja datos semi-estructurados
  - `resolveChoice()`: Resuelve conflictos de tipos
  - `unbox()`: Desempaqueta structs complejos

**Patrón de Job de Glue:**
```python
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'PARAM1'])
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# [ETL logic aquí]

job.commit()  # Marca job como exitoso en Catalog
```

**Optimizaciones:**
- **Pushdown Predicates**: Filtrar en lectura reduce datos procesados
- **Partition Pruning**: Leer solo particiones necesarias
- **Columnar Formats**: Parquet/ORC reducen IO 10x vs CSV

**Uso Real:**
Script Glue orquestado por Step Functions o EventBridge (CloudWatch Events) para procesamiento automático al detectar nuevos archivos en S3.

---
**Autor:** Luis J. Raigoso V. (LJRV)

Ejemplo de script de Glue (PySpark) para leer CSVs crudos de S3, limpiar/transformar y escribir en formato Parquet particionado por mes. Guarda este script como `glue_job.py` y súbelo a un Job de Glue.

In [ ]:
glue_job_script = r'''
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as F

args = getResolvedOptions(sys.argv, ['JOB_NAME','BUCKET','PREFIX_RAW','PREFIX_CURATED'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

raw_path = f